# Семинар 8 — Создание кастомного токенизатора для русскоязычного корпуса

-----

## Цель семинара

Сегодня мы создадим эффективный токенизатор для русского языка, реализовав  алгоритм BPE (Byte-Pair Encoding) используя корпус рускоязычной википедии. Мы заменим базовый токенизатор по схеме ReTok как это делали оптимизировав токенизацию на русском для LLM.

### Что мы построим:
- **BPE токенизатор** с нуля
- Корпус RU-Wiki
- **Unigram LM токенизатор** с библиотекой tokenizers
- Retok-замену токенизатора у выбранной LLM

### Ключевые навыки:
- Понимание алгоритмов subword токенизации
- Корректная работа с tokenizers
- Безопасная замена токенизатора в LLM
- Минимальный fine-tuning только эмбеддингов для стабилизации качества.

### Почему это важно для русского языка:
- Русские слова в среднем длиннее английских
- Богатая морфология (падежи, спряжения)
- Многие LLM используют неоптимальную токенизацию для русского так как изначально обучаются на английском
- Правильная токенизация может сократить длину последовательности и увеличить эффективность

-----

## 0. Установка и импорт библиотек

In [ ]:
# Установка необходимых библиотек
!pip install sentencepiece
!pip install tokenizers
!pip install pymorphy3
!pip install datasets
!pip install transformers
!pip install regex  # Поддержка Unicode в регулярках

In [ ]:
# Импорты
import os
import json
import math
import regex as re  # Вместо стандартного re для лучшей поддержки Unicode
import unicodedata
import collections
from typing import List, Dict, Tuple, Optional, Set
from dataclasses import dataclass
import warnings
warnings.filterwarnings('ignore')

# Обработка данных
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# Морфология
import pymorphy3

# Визуализация
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')

# Для сравнения с существующими токенизаторами
from transformers import AutoTokenizer
from transformers.trainer_utils import set_seed
import sentencepiece as spm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers

print("Все библиотеки импортированы")

# Создаём директории
os.makedirs('tokenizers', exist_ok=True)
os.makedirs('data', exist_ok=True)

Все библиотеки импортированы


## 1. Загрузка и подготовка русского корпуса

Для обучения токенизатора нам нужен представительный корпус русского текста.

In [ ]:
# Загружаем русский корпус википедии
from datasets import load_dataset
import os, re, unicodedata, hashlib

SNAPSHOT = "20231101.ru"      # можно заменить на другой доступный снапшот ru при желании
STREAMING = True              # stream=True не тянет весь датасет на диск
MAX_DOCS = 300_000            # сколько абзацев сохранить (поставьте None, чтобы не ограничивать)
MIN_LEN, MAX_LEN = 100, 4000  # длина абзаца в символах
OUT_TXT = "wiki_ru_corpus.txt"
OUT_ATTR = "wiki_ru_attribution.tsv"

# === 2) Загрузка RU Википедии (HF wikimedia/wikipedia) ===
# Поле 'text' уже очищено от разметки; есть также 'title' и 'url'. См. карточку датасета.  :contentReference[oaicite:2]{index=2}
ds = load_dataset("wikimedia/wikipedia", SNAPSHOT, split="train", streaming=STREAMING)

def normalize_text(s: str) -> str:
    s = unicodedata.normalize("NFKC", s)
    s = re.sub(r"[ \t\r\f\v]+", " ", s)
    s = re.sub(r"\n{3,}", "\n\n", s)
    return s.strip()

def iter_paragraphs(article_text: str):
    # грубо бьём на абзацы по пустой строке; можно усложнить при желании
    for p in re.split(r"\n{2,}", article_text):
        p = p.strip()
        if MIN_LEN <= len(p) <= MAX_LEN:
            yield p

# === 3) Потоковая выгрузка: пишем корпус + атрибуцию ===
os.makedirs(os.path.dirname(OUT_TXT) or ".", exist_ok=True)
seen = set()
n_written = 0

with open(OUT_TXT, "w", encoding="utf-8") as fo, \
     open(OUT_ATTR, "w", encoding="utf-8") as fa:
    fa.write("title\turl\n")
    for ex in ds:
        text = normalize_text(ex.get("text", "") or "")
        if not text:
            continue
        title = (ex.get("title") or "").replace("\t", " ").strip()
        url = (ex.get("url") or "").strip()

        # режем на абзацы и делаем простую дедупликацию по hash первых 1000 символов
        for para in iter_paragraphs(text):
            h = hashlib.sha1(para[:1000].encode("utf-8")).hexdigest()
            if h in seen:
                continue
            seen.add(h)
            fo.write(para + "\n")
            fa.write(f"{title}\t{url}\n")
            n_written += 1
            if MAX_DOCS and n_written >= MAX_DOCS:
                break
        if MAX_DOCS and n_written >= MAX_DOCS:
            break

print(f"Готово: записано {n_written} абзацев в {OUT_TXT}")
print(f"Атрибуция: {OUT_ATTR} (title, url на статью)")

# === 4) Быстрая проверка корпуса ===
# Соберём пару статистик: средняя длина абзаца и пример
import io, statistics, random
with open(OUT_TXT, "r", encoding="utf-8") as f:
    lines = [next(f).rstrip() for _ in range(min(1000, n_written))]  # первые 1000
avg_len = statistics.mean(len(s) for s in lines) if lines else 0
print(f"Пример: {lines[0][:180]}…")
print(f"Средняя длина первых 1000 абзацев: {avg_len:.1f} символов")

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

Готово: записано 300000 абзацев в wiki_ru_corpus.txt
Атрибуция: wiki_ru_attribution.tsv (title, url на статью)
Пример: Литва́ ( ), официальное название — Лито́вская Респу́блика () — государство, расположенное в Северной Европе. Площадь — км2. Протяжённость с севера на юг — 280 км, а с запада на вос…
Средняя длина первых 1000 абзацев: 264.0 символов


In [ ]:
OUT_TXT

'wiki_ru_corpus.txt'

## 2. Анализ особенностей русского языка

Прежде чем создавать токенизатор, проанализируем специфику русского языка.

In [ ]:
texts = lines
# Анализ символов в корпусе
def analyze_charset(texts: List[str]):
    """Анализ используемых символов в корпусе"""
    char_freq = collections.Counter()

    for text in tqdm(texts[:1000], desc="Анализ символов"):
        char_freq.update(text)

    # Категоризация символов
    categories = {
        'cyrillic': [],
        'latin': [],
        'digits': [],
        'punctuation': [],
        'spaces': [],
        'other': []
    }

    for char, freq in char_freq.items():
        if 'а' <= char <= 'я' or 'А' <= char <= 'Я' or char in 'ёЁ':
            categories['cyrillic'].append((char, freq))
        elif 'a' <= char <= 'z' or 'A' <= char <= 'Z':
            categories['latin'].append((char, freq))
        elif char.isdigit():
            categories['digits'].append((char, freq))
        elif char.isspace():
            categories['spaces'].append((char, freq))
        elif unicodedata.category(char).startswith('P'):
            categories['punctuation'].append((char, freq))
        else:
            categories['other'].append((char, freq))

    return char_freq, categories

char_freq, char_categories = analyze_charset(texts)

print("Распределение символов по категориям:")
for category, chars in char_categories.items():
    total_freq = sum(freq for _, freq in chars)
    print(f"  {category:12} : {len(chars):3} уникальных, {total_freq:8} вхождений")

# Топ символов
print("\nТоп-20 самых частых символов:")
for char, freq in char_freq.most_common(20):
    display_char = repr(char) if char.isspace() else char
    print(f"  '{display_char}' : {freq:6} раз")

Анализ символов:   0%|          | 0/1000 [00:00<?, ?it/s]

Распределение символов по категориям:
  cyrillic     :  63 уникальных,   210144 вхождений
  latin        :  49 уникальных,     1891 вхождений
  digits       :  10 уникальных,     6319 вхождений
  punctuation  :  21 уникальных,    10098 вхождений
  spaces       :   1 уникальных,    35393 вхождений
  other        :  17 уникальных,      192 вхождений

Топ-20 самых частых символов:
  '' '' :  35393 раз
  'о' :  22342 раз
  'и' :  17545 раз
  'е' :  16474 раз
  'а' :  15793 раз
  'н' :  14124 раз
  'с' :  13052 раз
  'т' :  11598 раз
  'р' :  10683 раз
  'в' :   9610 раз
  'л' :   8618 раз
  'к' :   6773 раз
  'м' :   6222 раз
  'д' :   5798 раз
  'п' :   4766 раз
  'у' :   4393 раз
  'ы' :   4254 раз
  'я' :   4231 раз
  'г' :   3591 раз
  ',' :   3489 раз


In [ ]:
# Морфологический анализ
morph = pymorphy3.MorphAnalyzer()

def analyze_morphology(texts: List[str], sample_size: int = 100):
    """Анализ морфологического разнообразия"""

    word_forms = collections.defaultdict(set)

    for text in tqdm(texts[:sample_size], desc="Морфологический анализ"):
        words = re.findall(r'\b[а-яёА-ЯЁ]+\b', text)

        for word in words:
            parsed = morph.parse(word.lower())[0]
            normal_form = parsed.normal_form
            word_forms[normal_form].add(word.lower())

    # Статистика
    forms_counts = [len(forms) for forms in word_forms.values()]

    print("Морфологическое разнообразие:")
    print(f"  Уникальных лемм: {len(word_forms)}")
    print(f"  Всего словоформ: {sum(forms_counts)}")
    print(f"  Среднее форм на лемму: {np.mean(forms_counts):.2f}")
    print(f"  Максимум форм: {max(forms_counts)}")

    # Примеры богатой морфологии
    print("\nПримеры слов с множеством форм:")
    rich_morphology = sorted(word_forms.items(), key=lambda x: len(x[1]), reverse=True)[:5]

    for lemma, forms in rich_morphology:
        print(f"  {lemma}: {', '.join(list(forms)[:10])}{'...' if len(forms) > 10 else ''}")

    return word_forms

word_forms = analyze_morphology(texts)

Морфологический анализ:   0%|          | 0/100 [00:00<?, ?it/s]

Морфологическое разнообразие:
  Уникальных лемм: 1418
  Всего словоформ: 2023
  Среднее форм на лемму: 1.43
  Максимум форм: 10

Примеры слов с множеством форм:
  литовский: литовском, литовскими, литовская, литовских, литовскую, литовской, литовские, литовское, литовский, литовского
  который: которого, который, которое, которую, которая, которые, которым, которой, которых
  польский: польский, польское, польскому, польским, польского, польскими, польской, польская, польские
  страна: страны, страной, стране, стран, страну, странах, страна, странами
  советский: советского, советскими, советскую, советской, советский, советская, советские, советским


## 3. Реализация BPE (Byte-Pair Encoding) токенизатора

BPE - самый популярный алгоритм для GPT-подобных моделей. Реализуем его с нуля.

In [ ]:
class BPETokenizer:
    """Byte-Pair Encoding токенизатор для русского языка"""

    def __init__(self, vocab_size: int = 10000):
        self.vocab_size = vocab_size
        self.word_freq = collections.Counter()
        self.vocab = {}
        self.merges = []

        # Специальные токены
        self.special_tokens = {
            '<PAD>': 0,
            '<UNK>': 1,
            '<BOS>': 2,
            '<EOS>': 3,
            '<MASK>': 4
        }
        self.next_id = len(self.special_tokens)

    def pre_tokenize(self, text: str) -> List[str]:
        """Предварительная токенизация на слова"""
        # Добавляем специальный символ для начала слова
        text = re.sub(r'\s+', ' ', text)
        words = text.split()
        return ['▁' + word for word in words]  # ▁ обозначает начало слова

    def get_word_frequencies(self, texts: List[str]) -> Dict[str, int]:
        """Подсчёт частот слов в корпусе"""
        word_freq = collections.Counter()

        for text in tqdm(texts, desc="Подсчёт частот"):
            words = self.pre_tokenize(text.lower())
            word_freq.update(words)

        return word_freq

    def get_pair_frequencies(self, word_freq: Dict[str, int]) -> Dict[Tuple[str, str], int]:
        """Подсчёт частот пар символов"""
        pair_freq = collections.Counter()

        for word, freq in word_freq.items():
            # Разбиваем слово на символы
            chars = list(word)

            # Считаем пары
            for i in range(len(chars) - 1):
                pair = (chars[i], chars[i + 1])
                pair_freq[pair] += freq

        return pair_freq

    def merge_pair(self, word_freq: Dict[str, int], pair: Tuple[str, str]) -> Dict[str, int]:
        """Слияние наиболее частой пары"""
        new_word_freq = {}
        merged = pair[0] + pair[1]

        for word, freq in word_freq.items():
            # Заменяем пару на слитый токен
            new_word = word.replace(pair[0] + ' ' + pair[1], merged)

            # Обновляем представление слова
            chars = []
            i = 0
            while i < len(word):
                if i < len(word) - len(pair[1]) and word[i:i+len(pair[0])] == pair[0] and word[i+len(pair[0]):i+len(pair[0])+len(pair[1])] == pair[1]:
                    chars.append(merged)
                    i += len(pair[0]) + len(pair[1])
                else:
                    chars.append(word[i])
                    i += 1

            new_word = ' '.join(chars) if len(chars) > 1 else chars[0]
            new_word_freq[new_word] = freq

        return new_word_freq

    def train(self, texts: List[str], min_freq: int = 2):
        """Обучение BPE токенизатора"""
        print("Обучение BPE токенизатора...")

        # 1. Получаем частоты слов
        word_freq = self.get_word_frequencies(texts)

        # 2. Фильтруем редкие слова
        word_freq = {w: f for w, f in word_freq.items() if f >= min_freq}

        # 3. Инициализируем словарь символами
        for word in word_freq:
            for char in word:
                if char not in self.vocab:
                    self.vocab[char] = self.next_id
                    self.next_id += 1

        # 4. Разбиваем слова на символы
        word_splits = {word: list(word) for word in word_freq}

        # 5. Обучаем merges
        pbar = tqdm(total=self.vocab_size - len(self.vocab), desc="Learning merges")

        while len(self.vocab) < self.vocab_size:
            # Находим самую частую пару
            pair_freq = collections.Counter()

            for word, freq in word_freq.items():
                splits = word_splits[word]
                for i in range(len(splits) - 1):
                    pair = (splits[i], splits[i + 1])
                    pair_freq[pair] += freq

            if not pair_freq:
                break

            # Самая частая пара
            best_pair = pair_freq.most_common(1)[0][0]
            merged = best_pair[0] + best_pair[1]

            # Сохраняем merge rule
            self.merges.append(best_pair)
            self.vocab[merged] = self.next_id
            self.next_id += 1

            # Обновляем разбиения
            for word in word_splits:
                splits = word_splits[word]
                new_splits = []
                i = 0

                while i < len(splits):
                    if i < len(splits) - 1 and (splits[i], splits[i + 1]) == best_pair:
                        new_splits.append(merged)
                        i += 2
                    else:
                        new_splits.append(splits[i])
                        i += 1

                word_splits[word] = new_splits

            pbar.update(1)

        pbar.close()
        print(f"✓ Обучение завершено. Размер словаря: {len(self.vocab)}")

    def tokenize(self, text: str) -> List[str]:
        """Токенизация текста"""
        words = self.pre_tokenize(text.lower())
        tokens = []

        for word in words:
            # Начинаем с посимвольного разбиения
            word_tokens = list(word)

            # Применяем learned merges
            for pair in self.merges:
                merged = pair[0] + pair[1]
                new_tokens = []
                i = 0

                while i < len(word_tokens):
                    if i < len(word_tokens) - 1 and word_tokens[i] == pair[0] and word_tokens[i + 1] == pair[1]:
                        new_tokens.append(merged)
                        i += 2
                    else:
                        new_tokens.append(word_tokens[i])
                        i += 1

                word_tokens = new_tokens

            tokens.extend(word_tokens)

        return tokens

    def encode(self, text: str) -> List[int]:
        """Кодирование текста в индексы"""
        tokens = self.tokenize(text)
        return [self.vocab.get(token, self.special_tokens['<UNK>']) for token in tokens]

    def decode(self, ids: List[int]) -> str:
        """Декодирование индексов в текст"""
        id_to_token = {v: k for k, v in self.vocab.items()}
        id_to_token.update({v: k for k, v in self.special_tokens.items()})

        tokens = [id_to_token.get(id, '<UNK>') for id in ids]
        text = ''.join(tokens)
        text = text.replace('▁', ' ').strip()
        return text

# Обучаем BPE токенизатор
bpe_tokenizer = BPETokenizer(vocab_size=1000)
bpe_tokenizer.train(texts[:100])  # Используем подвыборку для скорости

# Тестируем
test_text = "Тест токенизации"
tokens = bpe_tokenizer.tokenize(test_text)
encoded = bpe_tokenizer.encode(test_text)
decoded = bpe_tokenizer.decode(encoded)

print(f"\nТест BPE токенизатора:")
print(f"  Исходный: {test_text}")
print(f"  Токены: {tokens[:20]}..." if len(tokens) > 20 else f"  Токены: {tokens}")
print(f"  Encoded: {encoded[:20]}..." if len(encoded) > 20 else f"  Encoded: {encoded}")
print(f"  Decoded: {decoded}")

Обучение BPE токенизатора...


Подсчёт частот:   0%|          | 0/100 [00:00<?, ?it/s]

Learning merges:   0%|          | 0/942 [00:00<?, ?it/s]

✓ Обучение завершено. Размер словаря: 1000

Тест BPE токенизатора:
  Исходный: Тест токенизации
  Токены: ['▁т', 'е', 'ст', '▁то', 'к', 'ен', 'и', 'за', 'ции']
  Encoded: [105, 12, 68, 276, 14, 79, 11, 284, 355]
  Decoded: тест токенизации


## 4. Реализация WordPiece токенизатора

WordPiece используется в BERT и отличается от BPE критерием выбора пар для слияния.

In [ ]:
class WordPieceTokenizer:
    """WordPiece токенизатор (как в BERT)"""

    def __init__(self, vocab_size: int = 10000):
        self.vocab_size = vocab_size
        self.vocab = {}
        self.unk_token = '[UNK]'
        self.max_input_chars_per_word = 100

        # Специальные токены BERT-style
        self.special_tokens = {
            '[PAD]': 0,
            '[UNK]': 1,
            '[CLS]': 2,
            '[SEP]': 3,
            '[MASK]': 4
        }
        self.next_id = len(self.special_tokens)

    def train(self, texts: List[str], min_freq: int = 2):
        """Обучение WordPiece токенизатора"""
        print("Обучение WordPiece токенизатора...")

        # Подсчёт частот символов и подслов
        char_freq = collections.Counter()
        word_freq = collections.Counter()

        for text in tqdm(texts, desc="Сбор статистики"):
            words = text.lower().split()
            word_freq.update(words)
            for word in words:
                char_freq.update(word)

        # Инициализация словаря символами
        for char, freq in char_freq.items():
            if freq >= min_freq:
                self.vocab[char] = self.next_id
                self.next_id += 1

        # Добавляем начальные токены для подслов
        candidate_vocab = collections.Counter()

        for word, freq in word_freq.items():
            if freq < min_freq:
                continue

            # Генерируем все возможные подслова
            for i in range(len(word)):
                for j in range(i + 1, min(len(word) + 1, i + 10)):  # Ограничиваем длину
                    subword = word[i:j]

                    # Добавляем ## для не-начальных подслов
                    if i > 0:
                        subword = '##' + subword

                    candidate_vocab[subword] += freq

        # Отбираем топ кандидатов по частоте
        sorted_candidates = sorted(candidate_vocab.items(), key=lambda x: x[1], reverse=True)

        for token, freq in sorted_candidates:
            if len(self.vocab) >= self.vocab_size:
                break
            if token not in self.vocab:
                self.vocab[token] = self.next_id
                self.next_id += 1

        print(f"✓ Обучение завершено. Размер словаря: {len(self.vocab)}")

    def tokenize_word(self, word: str) -> List[str]:
        """Токенизация одного слова используя WordPiece"""
        if len(word) > self.max_input_chars_per_word:
            return [self.unk_token]

        # Жадный алгоритм максимального соответствия
        tokens = []
        start = 0

        while start < len(word):
            end = len(word)
            cur_substr = None

            while start < end:
                substr = word[start:end]

                # Добавляем ## для подслов
                if start > 0:
                    substr = '##' + substr

                if substr in self.vocab:
                    cur_substr = substr
                    break

                end -= 1

            if cur_substr is None:
                return [self.unk_token]

            tokens.append(cur_substr)
            start = end

        return tokens

    def tokenize(self, text: str) -> List[str]:
        """Токенизация текста"""
        words = text.lower().split()
        tokens = []

        for word in words:
            tokens.extend(self.tokenize_word(word))

        return tokens

    def encode(self, text: str) -> List[int]:
        """Кодирование в индексы"""
        tokens = self.tokenize(text)
        return [self.vocab.get(token, self.special_tokens['[UNK]']) for token in tokens]

    def decode(self, ids: List[int]) -> str:
        """Декодирование из индексов"""
        id_to_token = {v: k for k, v in self.vocab.items()}
        id_to_token.update({v: k for k, v in self.special_tokens.items()})

        tokens = [id_to_token.get(id, '[UNK]') for id in ids]

        # Склеиваем токены, убирая ##
        text = ''
        for token in tokens:
            if token.startswith('##'):
                text += token[2:]
            else:
                if text:
                    text += ' '
                text += token

        return text.strip()

# Обучаем WordPiece
wp_tokenizer = WordPieceTokenizer(vocab_size=1000)
wp_tokenizer.train(texts[:100])

# Тестируем
test_text = "Субсловная токенизация эффективна для морфологически богатых языков"
tokens = wp_tokenizer.tokenize(test_text)
encoded = wp_tokenizer.encode(test_text)
decoded = wp_tokenizer.decode(encoded)

print(f"\nТест WordPiece токенизатора:")
print(f"  Исходный: {test_text}")
print(f"  Токены: {tokens}")
print(f"  Encoded: {encoded[:20]}..." if len(encoded) > 20 else f"  Encoded: {encoded}")
print(f"  Decoded: {decoded}")

Обучение WordPiece токенизатора...


Сбор статистики:   0%|          | 0/100 [00:00<?, ?it/s]

✓ Обучение завершено. Размер словаря: 1000

Тест WordPiece токенизатора:
  Исходный: Субсловная токенизация эффективна для морфологически богатых языков
  Токены: ['с', '##у', '##б', '##сл', '##ов', '##на', '##я', 'то', '##к', '##ени', '##за', '##ци', '##я', '[UNK]', 'д', '##ля', '[UNK]', 'бо', '##г', '##ат', '##ых', 'язык', '##ов']
  Encoded: [22, 95, 139, 426, 100, 205, 88, 774, 85, 189, 410, 367, 88, 1, 30, 203, 1, 543, 109, 340]...
  Decoded: субсловная токенизация [UNK] для [UNK] богатых языков


## 5. Базовая модель и её токенизатор

Для быстрого демо используем лёгкую модель

In [ ]:
import os, re, time, math
from collections import Counter

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"
DTYPE = torch.float16 if DEVICE == "cuda" else torch.float32

old_tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=DTYPE, trust_remote_code=True
).to(DEVICE)
base_model.eval()



# Гигиена
if old_tok.pad_token_id is None:
    old_tok.pad_token = old_tok.eos_token
base_model.config.pad_token_id = base_model.config.eos_token_id


`torch_dtype` is deprecated! Use `dtype` instead!


## 6. Обучаем собственный токенайзер

Рекомендация (от разработчиков T-Pro): сохранить общий размер словаря, но расширить кириллицу за счёт малочастотных токенов.

In [ ]:
from transformers import AddedToken


CYR = re.compile(r"[А-Яа-яЁё]")

def iter_words(path):
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            for w in re.findall(r"[А-Яа-яЁё\-]{3,}", line):
                yield w.lower()

# Частотные слова из корпуса
cnt = Counter(iter_words(OUT_TXT))
word_candidates = [w for (w,c) in cnt.most_common(50000) if len(w) >= 6 and c >= 3]

# Длинные лексемы (ручной список под стиль ру-Вики)
manual_long = [
    "государственность","парламентаризм","конституционализм","федерализация","милитаризация",
    "административно-территориальный","правоприменение","регламентация","юрисдикция",
    "правосубъектность","правоспособность","дееспособность","кодификация",
    "кристаллография","радиоастрономия","электромагнетизм","спектрофотометрия","сверхпроводимость",
    "фотолюминесценция","нанотехнология","биоинформатика","геоморфология","палеоклиматология",
    "гидрогеология","сейсмоактивность","палеогеография","стратиграфия","субдукция","палеонтология",
    "искусствоведение","литературоведение","театроведение","источниковедение","лингвокультурология",
    "историография","социолингвистика","этнолингвистика","аксиоматизация","коммутативность",
    "ассоциативность","дифференцируемость","инвариантность","ортогональность","квазилинейность",
    "тензорный-анализ","микроэволюция","нейропластичность","нейродегенерация","иммуногистохимия",
    "антибиотикорезистентность","нейроанатомия","электрофизиология","нейрохимический",
    "макроэкономический","либерализация","монетизация","коммерциализация","национализация",
    "урбанизация","стратификация","демографический","кроссплатформенность","высокопроизводительный",
    "многопоточность","интероперабельность","микропроцессорный","криптографический",
    "видеокодирование","телекоммуникационный","библиографический","рецензирование",
    "диссертационный","цитируемость","импакт-фактор","наукометрия","наукоёмкость","междисциплинарный",
    "Россия", "российский", "данные", "модель", "металлургия", "ванадий",
    "ЕВРАЗ", "производство", "отчёт", "параметры", "семинар",
    "обучение", "инференс", "качественно", "генерация", "ядро", "система",
    "инструкция", "пример"
]

# Аффиксы/морфемы (их тоже полезно иметь)
affixes = [
    "изация","ический","ирование","ировать","ивативн","ость","ностью","ироватьс",
    "ователь","овательн","действи","безопасн","пользоват","отправк","кредитован",
    "самовывоз","международ","предприяти","организаци","по-","меж-","сверх-","пере-","пред-",
]

# Финальный пул кандидатов
MAX_NEW = 3000
pool, seen = [], set()
for w in word_candidates:
    if len(w) >= 8 and w not in seen and CYR.search(w):
        pool.append(w); seen.add(w)
    if len(pool) >= MAX_NEW: break
for w in manual_long:
    if w not in seen:
        pool.append(w); seen.add(w)

def build_added_tokens(str_list):
    # ВАЖНО: single_word=True, чтобы не матчить середину слова
    dedup, sset = [], set()
    for s in str_list:
        if s and s not in sset:
            dedup.append(s); sset.add(s)
    add = []
    for s in dedup:
        add.append(AddedToken(s, single_word=True, lstrip=False, rstrip=False))
        add.append(AddedToken(" " + s, single_word=True, lstrip=False, rstrip=False))
    return add

vocab_before = len(new_tok)
added_tokens = build_added_tokens(pool)
len(added_tokens)



6184

## 7. Retok-подмена: тёплая инициализация эмбеддингов + быстрое дообучение только эмбеддингов

Идея ReTok: заменить токенайзер, переинициализировать вход/выход из старых эмбеддингов, зафризить остальное и слегка дообучить.


In [ ]:
new_tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, trust_remote_code=True)
# if new_tok.pad_token_id is None:
#     new_tok.pad_token = new_tok.eos_token

vocab_before = len(new_tok)
n_added = new_tok.add_tokens(added_tokens, special_tokens=False)
print(f"Добавлено новых токенов: {n_added}")

# Новая модель под расширенный словарь
model_newtok = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=DTYPE, trust_remote_code=True
).to(DEVICE)
model_newtok.resize_token_embeddings(len(new_tok))  # ОБЯЗАТЕЛЬНО после add_tokens
model_newtok.config.pad_token_id = model_newtok.config.eos_token_id
model_newtok.eval()



Добавлено новых токенов: 6184


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(157849, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2

In [ ]:
def init_new_rows_from_old_subtokens(model, old_tokenizer, new_tokenizer, new_tokens, old_vocab_size):
    with torch.no_grad():
        emb = model.get_input_embeddings()          # nn.Embedding
        W = emb.weight                              # [V, d]
        d = W.shape[1]
        inited, skipped = 0, 0

        for t in new_tokens:
            new_id = new_tokenizer.convert_tokens_to_ids(t)
            if new_id is None or new_id < old_vocab_size:
                skipped += 1
                continue
            # разложим новый токен старым токенизатором
            sub = old_tokenizer.encode(t, add_special_tokens=False)
            if not sub:
                vec = torch.randn(d, dtype=W.dtype, device=W.device) * 0.02
            else:
                vec = W[sub].mean(dim=0)
            W[new_id] = vec
            inited += 1

        # если выходные эмбеддинги не "tied", скопируем им тоже
        out = model.get_output_embeddings()
        if out is not None and out.weight.shape[0] == W.shape[0]:
            out.weight.copy_(W)

    print(f"Инициализировано новых строк: {inited}; пропущено (существовали): {skipped}")

init_new_rows_from_old_subtokens(model_newtok, old_tok, new_tok, manual_long, len(old_tok))


Инициализировано новых строк: 96; пропущено (существовали): 0


In [ ]:
samples = [
    "Россия развивает металлургию и новые сплавы.",
    "RAG-система ускоряет поиск по документам ЕВРАЗ.",
    "Ретокенизация помогает сделать меньше токенов на русском. Например при обработке слова  нейроанатомия "
]

def tokens_per_char(tokenizer, text):
    ids = tokenizer.encode(text, add_special_tokens=False)
    return len(ids) / max(1, len(text))

print("--- До (old_tok) ---")
for s in samples:
    print(s, "→", tokens_per_char(old_tok, s), "toks/char")

print("\n--- После (tok) ---")
for s in samples:
    ids = new_tok.encode(s, add_special_tokens=False)
    print(s, "→", tokens_per_char(new_tok, s), "toks/char")

--- До (old_tok) ---
Россия развивает металлургию и новые сплавы. → 0.36363636363636365 toks/char
RAG-система ускоряет поиск по документам ЕВРАЗ. → 0.3617021276595745 toks/char
Ретокенизация помогает сделать меньше токенов на русском. Например при обработке слова  нейроанатомия  → 0.3333333333333333 toks/char

--- После (tok) ---
Россия развивает металлургию и новые сплавы. → 0.29545454545454547 toks/char
RAG-система ускоряет поиск по документам ЕВРАЗ. → 0.3404255319148936 toks/char
Ретокенизация помогает сделать меньше токенов на русском. Например при обработке слова  нейроанатомия  → 0.27450980392156865 toks/char


In [ ]:
w = "нейроанатомия"
print("old (no/space):", old_tok.convert_tokens_to_ids(w), old_tok.convert_tokens_to_ids(" " + w))
print("new (no/space):", new_tok.convert_tokens_to_ids(w), new_tok.convert_tokens_to_ids(" " + w))


old (no/space): None None
new (no/space): 157765 157766


In [ ]:
def align_special_tokens(tokenizer, model, prefer_chat_eos=True):
    # Qwen обычно использует `<|im_end|>` как маркер конца сегмента в чате
    im_end = tokenizer.convert_tokens_to_ids("<|im_end|>")
    eot    = tokenizer.convert_tokens_to_ids("<|endoftext|>")
    eos_id = im_end if (prefer_chat_eos and im_end is not None) else eot
    assert eos_id is not None, "Не найден подходящий EOS (<|im_end|> или <|endoftext|>)"

    # паддинг: добавим явный, если нет
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({"pad_token": "<|pad|>"})
        model.resize_token_embeddings(len(tokenizer))
    pad_id = tokenizer.pad_token_id or eos_id

    # Пропишем везде
    tokenizer.eos_token_id = eos_id
    model_newtok.config.eos_token_id = eos_id
    model_newtok.generation_config.eos_token_id = eos_id

    tokenizer.pad_token_id = pad_id
    model_newtok.config.pad_token_id = pad_id
    model_newtok.generation_config.pad_token_id = pad_id

align_special_tokens(new_tok, model_newtok, prefer_chat_eos=True)
print("EOS:", new_tok.eos_token, new_tok.eos_token_id, "| PAD:", new_tok.pad_token, new_tok.pad_token_id)


EOS: <|im_end|> 151645 | PAD: <|endoftext|> 151643


In [ ]:
for p in model_newtok.parameters():
    p.requires_grad_(False)

model_newtok.get_input_embeddings().weight.requires_grad_(True)
if model_newtok.get_output_embeddings() is not None:
    model_newtok.get_output_embeddings().weight.requires_grad_(True)

trainable = sum(p.numel() for p in model_newtok.parameters() if p.requires_grad)
total = sum(p.numel() for p in model_newtok.parameters())
print(f"Trainable params: {trainable:,} / {total:,}  (~{100*trainable/total:.4f}%)")

Trainable params: 141,432,704 / 499,330,816  (~28.3244%)


In [ ]:
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    DataCollatorForLanguageModeling, Trainer, TrainingArguments, set_seed
)
# Снова загрузим датасет, но теперь для дообучения
SNAPSHOT = "20231101.ru"
BLOCK_SIZE = 1024
MAX_TRAIN_BLOCKS = 2000
MAX_EVAL_BLOCKS  = 200

raw_stream = load_dataset("wikimedia/wikipedia", SNAPSHOT, split="train", streaming=True)

def yield_text(examples_iter):
    for ex in examples_iter:
        txt = ex.get("text", "")
        if isinstance(txt, str):
            t = txt.strip()
            if t:
                yield t

def tokenize_to_ids(text):
    return new_tok(text, truncation=True, max_length=BLOCK_SIZE, add_special_tokens=False)["input_ids"]

def to_blocks(ids_iter, block_size, max_blocks):
    buf = []
    n = 0
    for ids in ids_iter:
        buf.extend(ids)
        while len(buf) >= block_size:
            yield {"input_ids": buf[:block_size], "labels": buf[:block_size]}
            buf = buf[block_size:]
            n += 1
            if n >= max_blocks:
                return

train_iter = yield_text(iter(raw_stream))
train_ids_iter = (tokenize_to_ids(t) for t in train_iter)
train_data = list(to_blocks(train_ids_iter, BLOCK_SIZE, MAX_TRAIN_BLOCKS))

eval_iter = yield_text(iter(raw_stream))
eval_ids_iter = (tokenize_to_ids(t) for t in eval_iter)
eval_data = list(to_blocks(eval_ids_iter, BLOCK_SIZE, MAX_EVAL_BLOCKS))

train_ds = Dataset.from_list(train_data)
eval_ds  = Dataset.from_list(eval_data)
print("Train/Eval blocks:", len(train_ds), len(eval_ds))

collator = DataCollatorForLanguageModeling(tokenizer=new_tok, mlm=False)

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

Train/Eval blocks: 2000 200


In [ ]:

args = TrainingArguments(
    output_dir="retok_ru_embed_only",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=3e-3,
    warmup_steps=50,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=200,
    save_steps=1000,
    bf16=torch.cuda.is_available(),
    report_to=[],
)

trainer = Trainer(
    model=model_newtok,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=collator,
)

trainer.train()

metrics = trainer.evaluate()
# ppl = math.exp(metrics["eval_loss"])
print(f"Eval PPL: {ppl:.2f}")


Step,Training Loss,Validation Loss
200,18865.037500,16844.857422


OverflowError: math range error

In [ ]:
import time, math, torch
from transformers import GenerationConfig

# --- Константы и промпты ---
EVAL_PROMPTS = [
    "Официальное название Литвы",
    "НЭП - это",
    "Расскажи о пути из варяг в греки.",
]


GEN_CFG = GenerationConfig(
    max_new_tokens=160,
    do_sample=False,
    use_cache=True,
    repetition_penalty=1.1,
    no_repeat_ngram_size=3,
)

DEVICE = next(base_model.parameters()).device
base_model.eval(); model_newtok.eval()

# --- Вспомогательные функции ---
def _encode_chat(tok, text, device):
    # Рекомендованный путь для Instruct-моделей — chat template
    s = tok.apply_chat_template(
        [{"role": "user", "content": text}],
        tokenize=False, add_generation_prompt=True
    )
    enc = tok(s, return_tensors="pt")
    inputs = {"input_ids": enc["input_ids"].to(device)}
    if "attention_mask" in enc:
        inputs["attention_mask"] = enc["attention_mask"].to(device)
    return inputs

@torch.inference_mode()
def generate_one(model, tok, prompt, gen_cfg: GenerationConfig):
    inputs = _encode_chat(tok, prompt, model.device)
    out = model.generate(**inputs, generation_config=gen_cfg)
    # возвращаем только продолжение (без префикса чата)
    gen = out[0, inputs["input_ids"].shape[1]:]
    return tok.decode(gen, skip_special_tokens=True)

def tokens_per_char(tok, texts):
    vals=[]
    for t in texts:
        n = len(tok(t, add_special_tokens=False).input_ids)
        vals.append(n / max(1, len(t)))
    return sum(vals)/len(vals), vals

@torch.inference_mode()
def measure_latency(model, tok, texts, gen_cfg: GenerationConfig, warmup=1):
    prefill_ms, total_ms = [], []
    for t in texts:
        inputs = _encode_chat(tok, t, model.device)
        # прогрев
        for _ in range(warmup):
            _ = model(**inputs, use_cache=True)
            _ = model.generate(**inputs, generation_config=gen_cfg)
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        t0 = time.time(); _ = model(**inputs, use_cache=True);
        if torch.cuda.is_available(): torch.cuda.synchronize()
        t1 = time.time()
        prefill_ms.append((t1 - t0) * 1000)

        t0 = time.time(); _ = model.generate(**inputs, generation_config=gen_cfg)
        if torch.cuda.is_available(): torch.cuda.synchronize()
        t1 = time.time()
        total_ms.append((t1 - t0) * 1000)
    return sum(prefill_ms)/len(prefill_ms), sum(total_ms)/len(total_ms)

def reset_and_get_vram_mb():
    if not torch.cuda.is_available():
        return 0
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.synchronize()
    return int(torch.cuda.max_memory_allocated() / (1024*1024))

@torch.inference_mode()
def calc_ppl_on_texts(model, tok, texts):
    losses = []
    for txt in texts:
        enc = tok(txt, return_tensors="pt").to(model.device)
        out = model(**enc, labels=enc["input_ids"])
        losses.append(out.loss.item())
    mean_loss = sum(losses)/max(1, len(losses))
    return math.exp(mean_loss)

# Доля именно НОВЫХ (добавленных) токенов в кодировке текста
base_vocab = set(old_tok.get_vocab().keys())
ext_vocab  = new_tok.get_vocab()
new_only_ids = {i for s,i in ext_vocab.items() if s not in base_vocab}
def share_new_tokens(tok, text):
    ids = tok(text, add_special_tokens=False).input_ids
    return 0.0 if not ids else sum(int(i in new_only_ids) for i in ids) / len(ids)

# --- Подсчёты ---
# tokens/char
base_tpc, base_tpcs = tokens_per_char(old_tok, EVAL_PROMPTS)
ext_tpc,  ext_tpcs  = tokens_per_char(new_tok, EVAL_PROMPTS)

# Латентность и VRAM
_ = reset_and_get_vram_mb()
base_prefill, base_total = measure_latency(base_model, old_tok, EVAL_PROMPTS, GEN_CFG)
base_vram = reset_and_get_vram_mb()

_ = reset_and_get_vram_mb()
new_prefill, new_total   = measure_latency(model_newtok, new_tok, EVAL_PROMPTS, GEN_CFG)
new_vram = reset_and_get_vram_mb()


# Генерации
base_ans = [generate_one(base_model, old_tok, p, GEN_CFG) for p in EVAL_PROMPTS]
new_ans  = [generate_one(model_newtok, new_tok, p, GEN_CFG) for p in EVAL_PROMPTS]

# --- Сводка ---
print("=== Сводка (BASE vs EXT) ===")
print(f"tokens/char     base={base_tpc:.4f} | ext={ext_tpc:.4f}  (Δ {(ext_tpc/base_tpc-1)*100:+.1f}%)")
print(f"prefill (ms)    base={base_prefill:.1f} | ext={new_prefill:.1f}  (×{base_prefill/max(new_prefill,1e-6):.2f})")
print(f"total   (ms)    base={base_total:.1f}  | ext={new_total:.1f}   (×{base_total/max(new_total,1e-6):.2f})")
print(f"VRAM peak (MB)  base≈{base_vram} | ext≈{new_vram}")

print("Доля НОВЫХ токенов в промптах:")
for p in EVAL_PROMPTS:
    print(f"  {p[:36]}… -> {share_new_tokens(new_tok, p):.3f}")

print("\n— Примеры ответов —\n")
for i,(p,a0,a1,t0,t1) in enumerate(zip(EVAL_PROMPTS, base_ans, new_ans, base_tpcs, ext_tpcs),1):
    print(f"[{i}] PROMPT: {p}")
    print(f"tks/char: base={t0:.4f} | ext={t1:.4f}")
    print("BASE:", a0[:400].replace("\n"," ") + ("..." if len(a0)>400 else ""))
    print("EXT :", a1[:400].replace("\n"," ") + ("..." if len(a1)>400 else ""))
    print("-"*90)


=== Сводка (BASE vs EXT) ===
tokens/char     base=0.4494 | ext=0.4494  (Δ +0.0%)
prefill (ms)    base=27.0 | ext=37.8  (×0.71)
total   (ms)    base=3821.3  | ext=4586.1   (×0.83)
VRAM peak (MB)  base≈4404 | ext≈4404
Доля НОВЫХ токенов в промптах:
  Официальное название Литвы… -> 0.000
  НЭП - это… -> 0.000
  Расскажи о пути из варяг в греки.… -> 0.000

— Примеры ответов —

[1] PROMPT: Официальное название Литвы
tks/char: base=0.3077 | ext=0.3077
BASE: Извините, но я не могу предоставить информацию о официальном названии Литовской Республики. Вместо этого я могу предложить вам информацию о стране в целом:  1. Литва - это одна из самых маленьких стран Европы, расположенная на юге России.  2. Страна имеет исторический и культурный потенциал, но также面临许多挑战，包括气候变化、人口增长和经济不平等。  3. Летние месяцы в Литве обычно бывают от 6 до 8 месяцев, что делает её ...
EXT : роророракракракCrпедпор Брпредполагаетсяпредполагаетсяпредполагаетсяʥʥʥ Бр Бр Бр уравнений уравнений уравнений Бр Бр выпущена выпущена

## Дополнительные материалы

### Статьи:
- [Neural Machine Translation of Rare Words with Subword Units](https://arxiv.org/abs/1508.07909) - оригинальная BPE статья
- [SentencePiece](https://arxiv.org/abs/1808.06226) - унифицированная токенизация
- [Subword Regularization](https://arxiv.org/abs/1804.10959) - улучшение робастности
- https://huggingface.co/papers/2412.21140#:~:text=instruction,7B%2C%20showing
- https://habr.com/ru/articles/899242/
- https://arxiv.org/html/2410.04335v1

### Инструменты:
- [Tokenizers](https://github.com/huggingface/tokenizers) - быстрая реализация от HuggingFace
- [SentencePiece](https://github.com/google/sentencepiece) - Google's tokenizer
- [YouTokenToMe](https://github.com/VKCOM/YouTokenToMe) - BPE от VK

### Полезные ресурсы:
- [Tokenizer Arena](https://huggingface.co/spaces/Xenova/the-tokenizer-playground) - сравнение токенизаторов
- [Russian NLP](https://github.com/natasha/natasha) - инструменты для русского языка